Clean up user info data from myanimeList dataset

In [1]:
import pandas as pd
import numpy as np
import csv

In [2]:
user_df = pd.read_csv("my_anime_data_cleaned/user_data_v2.csv", sep=',')

In [3]:
anime_df = pd.read_csv("my_anime_data_cleaned/anime_data.csv", sep=",")

In [18]:
anime_df.iloc[0]

anime_index               0
anime_id               2366
TV                        1
ONA                       0
OVA                       0
                       ... 
completed_count         173
on_hold_count            31
dropped_count            75
plan_to_watch_count     544
total_count             884
Name: 0, Length: 87, dtype: object

In [4]:
rating = pd.read_csv("myanimedata/user_data/user_anime000000000000.csv", sep="\t")

In [5]:
rating.shape

(4003081, 16)

In [6]:
anime_df.shape

(13379, 87)

In [7]:
user_df.shape

(1123284, 10)

In [21]:
rating[["user_id", "anime_id", "review_score", "score", "status", "progress"]].iloc[100030]

user_id         -agonia-
anime_id           32262
review_score         NaN
score                NaN
status           dropped
progress             5.0
Name: 100030, dtype: object

In [21]:
user_df[user_df["user_id"] == "--chinoda--"]

,user_id,num_watching,num_completed,num_on_hold,num_dropped,num_plan_to_watch,num_days,mean_score
519533,--chinoda--,3,190,6,5,97,56.6,6.24


In [8]:
rating["status"].unique()

array(['completed', 'plan_to_watch', 'on_hold', 'watching', 'dropped'],
      dtype=object)

In [9]:
good_side = ["completed", "plan_to_watch", "watching"]

In [10]:
user_df["watched"] = None

In [14]:
user_df[user_df["user_id"] == "-------"].iloc[0]

user_index             42871
user_id              -------
num_watching              57
num_completed            401
num_on_hold               55
num_dropped                0
num_plan_to_watch        310
num_days               177.4
mean_score              6.53
watched                 None
Name: 42871, dtype: object

In [22]:
user_df.loc[user_df["user_id"] == "-agonia-", "watched"] += "voiture"

In [29]:
user_df[user_df["user_id"] == "-agonia-"]["watched"].isnull()

950737    True
Name: watched, dtype: bool

In [48]:
user_df.shape

(1123284, 10)

In [49]:
len(rating["anime_id"].unique())

13053

In [23]:
rating_matrix = np.empty((anime_df.shape[0], user_df.shape[0]),dtype=np.int8)

In [24]:
rating_matrix.fill(-1)

In [25]:
rating_matrix.shape

(13379, 1123284)

In [64]:
# user_df["watched"] = None
correction_index = 411663
# last_index = 411663
previous_id = 0
for i, row in rating.iloc[last_index:].iterrows() :
    status = row["status"]
    username = row["user_id"]
    score = row["score"]
     
    anime_id = row["anime_id"]


    if isinstance(score, float) :
        rate = 1 if score >= 6 else 0

        if username != previous_id :
            user_index = user_df.loc[user_df["user_id"] == username, "user_index"].iloc[0]
        
        anime_index = anime_df.loc[anime_df["anime_id"] == anime_id, "anime_index"].iloc[0]

        rating_matrix[anime_index, user_index] = rate


    if status in good_side or rate == 1 :
        if not user_df[user_df["user_id"] == username].iloc[0].values[-1] :
            user_df.loc[user_df["user_id"] == username, "watched"] = str(row["anime_id"]) + "|"

        else :
            old_value = user_df.loc[user_df["user_id"] == username]["watched"].iloc[0]

            new_value = str(old_value) + str(anime_id) + "|"

            user_df.loc[user_df["user_id"]== username, "watched"] = new_value

    print(i, end="\r")
    last_index = i

    previous_id = username


KeyboardInterrupt: 

In [44]:
user_df[user_df["user_id"] == "-khaled"].iloc[0, -1]

'1|10029|10030|1004|10055|10075|10083|10087|10092|1016|10162|10178|10216|1023|10259|10271|1029|1030|1033|1034|10379|10389|10408|10448|10460|1049|10500|10501|10589|10620|10622|10638|10643|10659|10686|1072|10721|1074|10800|10863|10893|1094|11061|11077|1109|11111|11113|1117|1118|1119|11241|1132|1142|11577|11633|11665|11701|11703|11739|11741|11771|11843|11887|1192|11981|12029|1205|12067|12069|1209|12189|12355|12413|12431|12445|12477|1251|12531|12565|12883|12893|1303|13125|1313|13169|13171|13173|13175|13253|1329|13331|13335|13469|13535|13599|136|13601|13629|1363|1364|1365|1367|137|1370|1373|1375|13759|1379|138|13807|13863|139|14075|14175|14189|14227|1430|14345|14347|14349|14353|14373|14397|14467|14513|1453|1462|14719|14807|14813|15021|15037|15039|1505|1506|1519|1520|1521|15227|15335|1535|15359|15417|15487|1564|1566|15689|1575|15809|15813|16|160|16001|16067|1614|16143|16149|16239|16241|16331|164|16498|16524|16528|16662|16664|16700|1674|16762|16774|16782|1681|1686|16866|16870|1689|16894|16916

In [65]:
user_df.to_csv("my_anime_data_cleaned/user_data_v2.csv", index=False, sep=",")

In [50]:
matrix_df = pd.DataFrame()

In [51]:
matrix_df["anime_index"] = anime_df["anime_index"]

In [52]:
matrix_df.drop(columns=["anime_id"], inplace=True)

In [66]:
def add_entry(row):
    user_index = user_df.loc[user_df["user_id"] == row["user_id"], "user_index"].iloc[0]
    
    anime_index = anime_df.loc[anime_df["anime_id"] == row["anime_id"], "anime_index"].iloc[0]

    score = row["score"]

    # rate = 1 if score >= 6 else 0
    rate = score

    if user_index not in matrix_df.columns :
        matrix_df[user_index] = -1

    matrix_df.loc[matrix_df["anime_index"] == anime_index, user_index] = rate

In [ ]:
for j, row in rating.iloc[correction_index+20000:last_index].iterrows() :
    add_entry(row)

    print("left : ", last_index - j, end="\r")

In [72]:
test_matrix_start_index = last_index
test_matrix_start_index

616798

In [ ]:
last_index = 616798
correction_index = 411663

In [68]:
matrix_df.shape

(13379, 574)

In [69]:
matrix_df.to_csv("my_anime_data_cleaned/rating_matrix_v1.csv", index=False, sep=",")